In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot as plt
from matplotlib import pyplot as plt

In [ ]:
X = np.load("../malaria/train_test_data/training_COL_X_all.npy")
y = np.load("../malaria/train_test_data/training_COL_y_all.npy")


In [ ]:
# plt.imshow(X[500])

In [ ]:
#convert y into OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(y.reshape(-1,1))

In [ ]:
y_encoded = ohe.transform(y.reshape(-1,1))# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")


In [ ]:
y_encoded_df=pd.DataFrame(y_encoded, columns = ohe.get_feature_names_out())

In [ ]:
y_encoded_df

In [ ]:
y_encoded.shape

# Balancing the Datasets

In [ ]:
from sklearn.utils import class_weight

In [ ]:
cw = y_encoded_df.sum().to_dict()
cw

In [ ]:
cwv = class_weight.compute_class_weight('balanced', 
                            classes = np.unique(y),
                            y=y)

In [ ]:
z = zip([int(i) for i in (np.arange(0,7))], 
            cwv)
np.unique(y_encoded)
[int(i) for i in (np.arange(0,7))]
weighting = dict(z)
weighting


In [ ]:
if X.ndim == 3:
    X_expanded = np.expand_dims(X,axis=3)
    image_shape = (X_expanded.shape[1], X_expanded.shape[2], X_expanded.shape[3])
    X=X_expanded
else:
    image_shape = (X.shape[1], X.shape[2], X.shape[3])

In [ ]:
len(X)

In [ ]:
X=X/255 #.shape
image_shape

In [ ]:
y_encoded.shape

In [ ]:
# from keras.layers import BatchNormalization

# ARCHITECTURE imports

# import tensorflow as tf
import numpy as np
from tensorflow.keras import  Sequential, layers, regularizers, metrics

# from keras.layers import BatchNormalization

#### 1. ARCHITECTURE

model = Sequential()

model.add(layers.Conv2D(filters=36, kernel_size=(5,5), input_shape=image_shape, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))


model.add(layers.Dropout(0.5))
# model.add(layers.Conv2D(filters=20, kernel_size=(3,3), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Conv2D(filters=10, kernel_size=(2,2), activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.01)))
model.add(layers.Conv2D(filters=20, kernel_size=(2,2), activation='relu'))

model.add(layers.MaxPooling2D(pool_size=(2, 2)))

#  model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

# Dropouts reduce overfitting by randomly turning neurons off during training.

model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
# binary = sigmoid
model.add(layers.Dense(7, activation='softmax'))

# COMPILATION imports

from tensorflow.keras import optimizers, callbacks

#### 2. COMPILATION

# adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
# adam = optimizers.Adam(learning_rate=0.1)
adam = optimizers.Adam()

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy',metrics.Recall()]
             )

model.summary()

In [ ]:
# FIT imports

from tensorflow.keras import callbacks


#### 3. FIT

es = callbacks.EarlyStopping(patience=15, restore_best_weights=True)

history = model.fit(X, y_encoded,
          batch_size=64, # Batch size -too small--> no generalization
          epochs=100,    #            -too large--> slow computations
          validation_split=0.2,
          callbacks=[es],
          class_weight = weighting,
          verbose=1)


In [ ]:
from matplotlib import colors 
def plot_loss(history, label, n):
    axs = plt.gca()
    ax2 = axs.twinx()
    # Use a log scale on y-axis to show the wide range of values.
    axs.semilogy(history.epoch, history.history['loss'],
               color='b', label='Train ' + label)
    axs.semilogy(history.epoch, history.history['val_loss'],
               color='r', label='Val ' + label,
               linestyle="--")
    axs.set_xlabel('Epoch')
    axs.set_ylabel('Loss')
    ax2.plot(history.epoch, history.history['recall'],
           color = 'g', label ='Recall' + label, linestyle = "-.")
    ax2.plot(history.epoch, history.history['accuracy'],
           color = 'y', label ='Recall' + label, linestyle = "-")

In [ ]:
plot_loss(history,"first run",'b')

In [ ]:
w = model.get_weights

In [ ]:
w[0]

In [ ]:
min(history.history['loss'])

In [ ]:
len(history.history['loss'])